In [2]:
import pandas as pd
import csv
import warnings
import os
from parse import *
warnings.filterwarnings("ignore")

In [3]:
filename = "Taxi_Trips_-_2019.csv"
df = pd.read_csv(filename)

with open('taxiDict.csv') as csv_file:
    reader = csv.reader(csv_file)
    mydict = dict(reader)

dictCompany = dict((v, int(k)) for k, v in mydict.items())

dfTaxi = preprocess(df, dictCompany)

In [4]:
# All Areas All Taxi
mkdir("allAreaAllTaxi")
mkdir("allAreaAllTaxi/outsideCity/")
mkdir("allAreaAllTaxi")
mkdir("allAreaAllTaxi/onlyCity/")

preCalc(dfTaxi, "allAreaAllTaxi/outsideCity/")
dfTaxiSub = dfTaxi[~((dfTaxi['pickupArea']==78) | (dfTaxi['dropArea']==78))]
preCalc(dfTaxiSub, "allAreaAllTaxi/onlyCity/")

In [5]:
# All Taxi Pickup Areas

mkdir("allTaxi")
mkdir("allTaxi/From")

for i in range(1,78):
    dir = "allTaxi/From/Area-{}/".format(i)
    mkdir(dir)

    sub = dfTaxi[dfTaxi['pickupArea']==i]
    dir = "allTaxi/From/Area-{}/outsideCity/".format(i)
    mkdir(dir)
    preCalc(sub, dir, True, False, True)

    sub = dfTaxiSub[dfTaxiSub['pickupArea']==i]
    dir = "allTaxi/From/Area-{}/onlyCity/".format(i)
    mkdir(dir)
    preCalc(sub, dir, True, False, False)

In [6]:
# All Taxi Dropoff Areas
# Percentage Coming From

mkdir("allArea")
mkdir("allTaxi")
mkdir("allTaxi/To")

for i in range(1,78):
    dir = "allTaxi/To/Area-{}/".format(i)
    mkdir(dir)

    sub = dfTaxi[dfTaxi['dropArea']==i]
    dir = "allTaxi/To/Area-{}/outsideCity/".format(i)
    mkdir(dir)
    preCalc(sub, dir, False, True, True)

    sub = dfTaxiSub[dfTaxiSub['dropArea']==i]
    dir = "allTaxi/To/Area-{}/onlyCity/".format(i)
    mkdir(dir)
    preCalc(sub, dir, False, True, False)

In [7]:
# All Areas
mkdir("allArea")

for i in range(55):
    dir = "allArea/Taxi-{}/".format(i)
    mkdir(dir)

    sub = dfTaxi[dfTaxi['company']==i]
    dir = "allArea/Taxi-{}/outsideCity/".format(i)
    mkdir(dir)
    preCalc(sub, dir)

    sub = dfTaxiSub[dfTaxiSub['company']==i]
    dir = "allArea/Taxi-{}//onlyCity/".format(i)
    mkdir(dir)
    preCalc(sub, dir)

In [8]:
# Taxi - Area (To/From) All Combinations
mkdir("allCombination")

for i in range(55):
    dir = "allCombination/Taxi-{}/".format(i)
    mkdir(dir)
    mkdir(dir+"From/")
    mkdir(dir+"To/")
    
    for j in range(1,78):
        dir = "allCombination/Taxi-{}/From/Area-{}".format(i,j)
        mkdir(dir)
        
        dir = "allCombination/Taxi-{}/From/Area-{}/onlyCity/".format(i,j)
        mkdir(dir)
        sub = dfTaxiSub[(dfTaxiSub['company']==i) & (dfTaxiSub['pickupArea']==j)]
        preCalc(sub, dir, True, False, False)

        dir = "allCombination/Taxi-{}/From/Area-{}/outsideCity/".format(i,j)
        mkdir(dir)
        sub = dfTaxi[(dfTaxi['company']==i) & (dfTaxi['pickupArea']==j)]
        preCalc(sub, dir, True, False, True)

        dir = "allCombination/Taxi-{}/To/Area-{}".format(i,j)
        mkdir(dir)
        
        dir = "allCombination/Taxi-{}/To/Area-{}/onlyCity/".format(i,j)
        mkdir(dir)
        sub = dfTaxiSub[(dfTaxiSub['company']==i) & (dfTaxiSub['dropArea']==j)]
        preCalc(sub, dir, False, True, False)

        dir = "allCombination/Taxi-{}/To/Area-{}/outsideCity/".format(i,j)
        mkdir(dir)
        sub = dfTaxi[(dfTaxi['company']==i) & (dfTaxi['dropArea']==j)]
        preCalc(sub, dir, False, True, True)